As of now i am just make a classification model in just one ipynb not in multiple modules. I will use pytorch for classification.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

loading the data and splitting the data into the train and the test set before doing that we will verify our image datasets if encounter any corroupted file or broken image file we will remove it.

In [2]:
import os
from PIL import Image

data_dir = "kagglecatsanddogs_3367a\PetImages"
classes = ["Cat","Dog"]

for cls in classes:
    cls_path = os.path.join(data_dir,cls)
    for img in os.listdir(cls_path):
        img_path=os.path.join(cls_path,img)
        try:
            with Image.open(img_path) as im:
                im.verify()
                
        except:
            print(f"this file is corroupted : {img_path}")
            os.remove(img_path)
        

c:\Users\Pallavi\OneDrive\Documents\computer vision\venv\lib\site-packages\PIL\TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


we fetched out a corroupted file out of the dataset now we can split our data. we will be using os as always 🗿 and shutil to make robust copy of data from one loc to other.

In [3]:
import os,shutil,random

root_dir = "kagglecatsanddogs_3367a\PetImages"
dest="Data"
classes = ["Cat","Dog"]
split=0.8

for cls in classes:
    images = os.listdir(os.path.join(root_dir,cls))
    random.shuffle(images)
    
    train_cut = int(len(images)*split)
    
    train_list = images[:train_cut]
    test_list = images[train_cut:]
    
    for phase,imgs in [("train",train_list),("test",test_list)]:
        out_dir = os.path.join(dest,phase,cls)
        os.makedirs(out_dir,exist_ok=True)
        for img in imgs:
            shutil.copy(
                os.path.join(root_dir,cls,img),
                os.path.join(out_dir,img)
            )
    

loading and tranforming the data for the image classification.

In [4]:
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

train_ds = datasets.ImageFolder("data/train",transform=transform)
test_ds = datasets.ImageFolder("data/test",transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader   = DataLoader(train_ds, batch_size=32, shuffle=False)

train_ds

Dataset ImageFolder
    Number of datapoints: 24144
    Root location: data/train
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

I SPLITTED THE DATA INTO TRAIN AND THE TEST SET AND ALSO REDUCED ALL THE IMAGE SIZE TO AVOID ANY BUG. NOW I AM GONNA MAKE A SIMPLE CNN MODEL WHICH GONNA CLASSIFY THE GIVEN PICTURE IS DOG OR OF CAT.

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, 3, activation='relu' , input_shape=(128,128,3)),
    MaxPooling2D(2),
    
    Conv2D(64,3,activation='relu'),
    MaxPooling2D(2),
    
    Conv2D(128,3,activation='relu'),
    MaxPooling2D(2),
    
    Flatten(),
    Dense(128,activation='relu'),
    Dense(1,activation='sigmoid')
    
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    
)

model.summary()

c:\Users\Pallavi\OneDrive\Documents\computer vision\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break


torch.Size([32, 3, 128, 128]) torch.Size([32])


SO AS I CAN SEE HERE IF I WANT TO FIT THIS DATA FIRST OF ALL I NEED TO CONVERT WE HAVE CHW BUT WE NEED HWC AND WE ARE CONVERTING THIS TENSOR INTO NUMPY ARRAY BECAUSE KERAS EXPECT NUMPY ARRAYS.

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2   # 👈 BEST PRACTICE
)


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = train_datagen.flow_from_directory(
    'data/train',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 19316 images belonging to 2 classes.


In [9]:
val_gen = train_datagen.flow_from_directory(
    'data/train',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 4828 images belonging to 2 classes.


In [10]:
model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen,
    steps_per_epoch=100//32
)

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 68s 32s/step - accuracy: 0.5521 - loss: 1.2288 - val_accuracy: 0.4998 - val_loss: 0.8985
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 24s 12s/step - accuracy: 0.3958 - loss: 0.8419 - val_accuracy: 0.4998 - val_loss: 0.6963
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 11s/step - accuracy: 0.5000 - loss: 0.6912 - val_accuracy: 0.4986 - val_loss: 0.6933
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 11s/step - accuracy: 0.5938 - loss: 0.6922 - val_accuracy: 0.4998 - val_loss: 0.6964
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 11s/step - accuracy: 0.4583 - loss: 0.7021 - val_accuracy: 0.4998 - val_loss: 0.6934


AS PROCESSING 24K IMAGES WAS RAM HEAVY I REDUCED SIZE BUT THIS MODEL IS BULLSHIT BECAUSE WE ARE GETTING ACCURACY SOMEHERE 0.5 MEANS 1/2 CHANCE THAT PRIDICTION CAN BE TRUE THIS MIGHT bE BCZ OF SMALL TRAINING EX AND ALSO BCz OF THE SIMPLE MODEL. THIS MODEL IS HIGLY BIASED I GONNA PICK A BIGGER BETTER MODEL. I GONNA USE MOBILENET V2 FOR THE SAME ISSUE.

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [18]:
base_model = MobileNetV2(
    input_shape=(128,128,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

# now we gonna make a custom head acc to out problem which we gonna make trainable to 
# learn better weights

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)


In [19]:
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [22]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=1000//32
)

Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.9536 - loss: 0.1242 - val_accuracy: 0.9453 - val_loss: 0.1353
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9476 - loss: 0.1187 - val_accuracy: 0.9482 - val_loss: 0.1321
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9466 - loss: 0.1387 - val_accuracy: 0.9495 - val_loss: 0.1229
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9536 - loss: 0.1259 - val_accuracy: 0.9515 - val_loss: 0.1214
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.9476 - loss: 0.1280 - val_accuracy: 0.9490 - val_loss: 0.1284
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.9435 - loss: 0.1285 - val_accuracy: 0.9495 - val_loss: 0.1292
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9335 - loss: 0.1566 - val_accuracy: 0.9513 - val_loss: 0.1161
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.9385 - loss: 0.1452 - val_accuracy: 0.9486 - val_loss:

In [32]:
from tensorflow.keras.preprocessing import image
import numpy as np

image_path = "Data/test/Dog/2.jpg"
img = image.load_img(image_path, target_size=(128,128))

img_array = image.img_to_array(img)
img_array.shape

img_array = np.expand_dims(img_array, axis=0)


In [ ]:
from tensorflow.keras.application.mobilenet_v2 import preprocess_input
ImageDataGenerator(preprocessing_function=preprocess_input)
img_array=preprocess_input(img_array)
pred = model.predict(img_array)
print(pred)